# 스파크 초기화

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                    .master('spark://spark-master:7077')\
                    .appName('spark-streaming-intro')\
                    .getOrCreate()

print(spark.version)

3.1.2


## 소스 : 스트리밍 데이터 수집
SparkSession에서 format이라는 스트리밍 소스를 지정하고 해당 구성을 제공하는 빌더 메소드인 readStream을 제공한다.    
- format : 소스 유형을 지정한다
- schema : 특정 소스 유형에 필수인 데이터 스트림에 대한 스키마를 제공할 수 있게 한다
- mode : 각 소스의 조정 가능한 파라미터

spark.readStream이 호출되면 DataStreamBuilder 인스턴스를 생성한다. 

- load(...)는 빌더에 제공된 옵션의 유효성을 확인하고 스트리밍 데이터 프레임을 반환한다.

배치 작업인 read/write에 대응되며 writeStream은 프로세스에 필요한 출력 싱크 및 출력 모드를 지정한다.

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType

schema = StructType([
    StructField("host", StringType(), True), # Name, Type, Null 
    StructField("timestamp", TimestampType(), True),
    StructField("request", StringType(), True),
    StructField("http_reply", IntegerType(), True),
    StructField("bytes", LongType(), True)
])

In [10]:
fileStream = spark.readStream.format('json')\
                             .schema(schema)\
                             .option("mode", "DROPMALFORMED")\
                             .load("/opt/workspace/datasets/NASA-weblogs/nasa_dataset_july_1995/*.json")

In [12]:
fileStream

DataFrame[host: string, timestamp: timestamp, request: string, http_reply: int, bytes: bigint]

스트리밍 DataFrame을 만들면 스트림이 구체화될 때까지 실제로는 사용되거나 처리되는 데이터가 없다.    
Query가 필요하다.

## 사용 가능한 소스
### file
json, orc, parquet, csv, text, textFile     
파일 기반 스트리밍 소스로 파일 시스템에서 경로를 모니터링하고 그안에 배치된 파일을 사용한다.   
### socket ( 테스트용 )
소켓 연결으로 텍스트 데이터를 읽는다. 엔드 포인트간 내결함성 보장하지는 않아서 테스트ㅠ용이다.
### kafka 
카프카 소비자를 생성해서 연결한다. 브로커 버전 0.10.0 이상을 지원한다.
### rate
초당 지정된 수의 행으로 스트림을 생성하고 마찬가지로 테스트 용이다.

## 데이터프레임 API에서의 스트리밍 API 제한
스트리밍 컨텍스트에서는 표준 데이터프레임과 데이터셋 API에서 제공하는 일부의 작업이 의미가 없다.   
아래는 스트림 상에서 직접적으로 지원되지 않는 API 작업이다.   
- count ( 스트림을 계산할 수는 없지만, 분당 수신한 메시지를 count하거나 특정 유형의 기기수를 count할 수는 있다 )
- show ( 'console' 싱크로 화면에 데이터를 출력할 수 있다 )
- describe
- limit
- take(n)
- distinct ( dropDuplicates("<key-column>") 로 복제본을 제거할 수 있다 )
- foreach ( foreach 싱크가 있다 )
- sort ( 출력 모드를 'complete' 를 사용한 쿼리에서는 집계된 데이터에 대한 정렬은 정의할 수 있다 )
- 누적된 여러 개의 집계

## 싱크: 결과 데이터 출력
스트림 시작 전에 출력 데이터의 위치(where)와 방법(how)을 정의해야 한다.
- where ( 스트리밍 데이터의 수신 측 )
- how ( 최종 레코드 처리 방법 )
스트리밍 데이터프레임 또는 데이터셋에서 writeStream을 호출해서 스트림을 구체화한다.    
writeStream을 호출하면 빌더 인스턴스인 DataStreamWriter가 생성되고 출력 동작을 구성하는 메소드를 제공한다. 

In [18]:
query = fileStream.writeStream.format("json")\
                              .queryName("json-writer")\
                              .outputMode("append")\
                              .option("path", "/opt/workspace/datasets/target/data")\
                              .option("checkpointLocation", "/opt/workspace/datasets/target/checkpoint")\
                              .trigger(processingTime='5 seconds')\
                              .start()

In [19]:
query

## 출력 싱크 포맷
### console ( 디버깅 )
트리거가 있을 때마다 표준 출력으로 출력되는 싱크다. 추가 및 완료 출력 모드가 모두 지원된다.
### file
csv, hive, json, orc, parquet, avro, text    
출력을 디렉터리에 저장한다. 
### kafka
하나 이상의 Topic에 저장할 수 있는 프로듀서 싱크다.
### memory ( 디버깅 )
메모리 내 테이블로 메모리에 저장된다. 추가 및 완료 출력 모두가 모두 지원된다.
### foreach
한 번에 한 요소씩 스트림 컨텐츠에 접근하기 위한 프로그래밍 인터페이스를 제공한다.
### foreachBatch
각 기본 마이크로배치에 해당하는 완전한 데이터프레임에 액세스 할 수 있는 프로그래밍 방식 싱크 인터페이스를 제공한다. 

### 출력 싱크 모드
### append ( 기본값 )
출력 스트림에 최종 레코드만 추가한다. 추가된 행이 절대 변경되지 않는 쿼리에 대해서만 지원된다. 각 행이 한 번만 출력되도록 보장한다.
<br>append 시멘틱</br>
스트리밍 쿼리에 집계가 포함된 경우 마지막 정의가 중요하지 않게 된다. 새 수신 레코드는 사용된 집계 기준을 준수할 때 기존 집계 값을 변경할 수 있다. 값이 최종임을 알때 까지 append를 사용해서 레코드를 출력할 수 없다. 따라서 집계 쿼리와 조합하여 append 출력 모드를 사용하는 것은 이벤트 시간을 사용하여 집계가 표현되는 쿼리로 제한되며 워터마크를 정의한다. 워터마크 만료 시 append가 이벤트를 출력하므로 집계된 값을 변경할 수 있는 새 레코드가 없다. 
### update
마지막 trigger 이후의 새롭게 추가되거나 업데이트된 레코드가 싱크로 출력된다. 
### complete
모든 트리거 후에 전체 결과가 싱크로 출력된다. 집계 쿼리에 대해 지원된다. 

## queryName
queryName을 사용하면 일부 싱크에서 사용하고 스파크 콘솔의 작업 설명에 표시되는 쿼리 이름을 제공할 수 있다.
## option
option 메소드로 소스에서 처럼 필요한 만큼 특정 키-값으로 스트림을 커스텀할 수있다.    
option의 대안으로 options를 사용할 수도 있다.
## trigger
결과를 생성할 빈도를 지정할 수 있다. 
- 시간 간격 ( ProcessingTime )
- 하나의 샷 배치 작업으로 테스트에 유용하다 ( Once )
- continuous 엔진으로 전환해서 실험용으로 사용 ( Continuous )
## start()
전체 잡 디스트립션을 스트리밍 연산으로 구체화하고 내부 스케줄링 프로세스를 시작하여 소스에서 데이터를 소비하고 처리해서 싱크로 생성한다. 각 쿼리 개별 수명 주기를 관리하는 핸들인 StreamingQuery 객체를 반환한다.